In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

C:\Anaconda\envs\py36gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 346109123016907044
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11054556775
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7597009084736639112
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0001:00:00.0, compute capability: 3.7"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 11054556775
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7665372117181562774
physical_device_desc: "device: 1, name: Tesla K80, pci bus id: 0002:00:00.0, compute capability: 3.7"
]


In [3]:
def set_gpus():
    gpus=tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu,True)
            logical_gpus=tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), " physical GPUs, ",len(logical_gpus)," Logical GPUs")
        except RuntimeError as e:
            print("Error: ",e)

In [4]:
def build_model():
    #strategy=tf.distribute.MirroredStrategy()
    #https://github.com/tensorflow/tensorflow/issues/21470#issuecomment-422506263
    #cross_tower_ops = tf.distribute.AllReduceCrossDeviceOps(
    #'hierarchical_copy', num_packs=num_gpus)
    #strategy = tf.distribute.MirroredStrategy(cross_tower_ops=cross_tower_ops)
    #with strategy.scope():
    
    with tf.device('/GPU:0'):
        
        model=tf.keras.Sequential([
            layers.Dense(64,activation="relu",input_shape=(32,)),
            layers.Dense(64,activation="relu"),
            layers.Dense(10)
        ])
        model.compile(optimizer=tf.keras.optimizers.SGD(0.01),
                     loss=tf.keras.losses.mse,
                      metrics=['accuracy']
                     )
        
        return model

In [5]:
def create_data():                      #create 1000 training samples. 100 validation samples
    #create training data.
    X=np.random.random((1000,32))
    Y=np.random.random((1000,10))
    
    #generate some validation data
    X_val=np.random.random((100,32))
    Y_val=np.random.random((100,10))
    
    #generate test data
    X_test=np.random.random((100,32))
    Y_test=np.random.random((100,10))
    
    return (X,Y,X_val,Y_val,X_test,Y_test)

In [6]:
(x_tr,y_tr,x_val,y_val,x_tst,y_tst)=create_data()
tr_dataset=tf.data.Dataset.from_tensor_slices((x_tr,y_tr)).batch(32)
vl_dataset=tf.data.Dataset.from_tensor_slices((x_val,y_val)).batch(32)
tst_dataset=tf.data.Dataset.from_tensor_slices((x_tst,y_tst)).batch(32)


In [7]:
model=build_model()

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                2112      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 6,922
Trainable params: 6,922
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(tr_dataset,epochs=10,validation_data=vl_dataset)

Train for 32 steps, validate for 4 steps
Epoch 1/10
32/32 [==============================] - 1s 25ms/step - loss: 0.2941 - accuracy: 0.0970 - val_loss: 0.1826 - val_accuracy: 0.1000
Epoch 2/10
32/32 [==============================] - 0s 2ms/step - loss: 0.1493 - accuracy: 0.0960 - val_loss: 0.1154 - val_accuracy: 0.0800
Epoch 3/10
32/32 [==============================] - 0s 2ms/step - loss: 0.1118 - accuracy: 0.0960 - val_loss: 0.0992 - val_accuracy: 0.1000
Epoch 4/10
32/32 [==============================] - 0s 3ms/step - loss: 0.1025 - accuracy: 0.0960 - val_loss: 0.0956 - val_accuracy: 0.1200
Epoch 5/10
32/32 [==============================] - 0s 2ms/step - loss: 0.0999 - accuracy: 0.1010 - val_loss: 0.0945 - val_accuracy: 0.1200
Epoch 6/10
32/32 [==============================] - 0s 2ms/step - loss: 0.0989 - accuracy: 0.0990 - val_loss: 0.0939 - val_accuracy: 0.1300
Epoch 7/10
32/32 [==============================] - 0s 2ms/step - loss: 0.0982 - accuracy: 0.1030 - val_loss: 0.0935 -

In [10]:
#evaluate the model
model.evaluate(tst_dataset)

4/4 [==============================] - 0s 2ms/step - loss: 0.0958 - accuracy: 0.1000


[0.09583684988319874, 0.1]